# 4 Pre-Processing and Training Data<a id='4_Pre-Processing_and_Training_Data'></a>

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Pre-Processing and Training Data](#4_Pre-Processing_and_Training_Data)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load Data](#4.4_Load_Data)
  * [4.5 Explore the Data](#4.5_Explore_Data)
  * [4.6 Handle missing values](#4.6_missing)
    * [4.6.1 Total Charges missing values](#4.6.1_Replace)   
        * [4.6.1.1 Replace missing values](#4.6.1.1_Replace)      
  * [4.7 Encode categorical values](#4.7_Encoding)
    * [4.7.1 Replace 'Yes', 'No' valuess](#4.7.1_Replace_Yes_No) 
    * [4.7.2 Encode ordinal variables](#4.7.2_ordinal)     
    * [4.7.3 Encode other Categorical Variables](#4.7.3_Other)    
  * [4.8 Scaling numerical featuresl](#4.8_Scaling)
  * [4.9 Create new featuresl](#4.9_Create)   
  * [4.10 Feature selectionl](#4.10_Feature)   
  * [4.11 Train/Test Splitl](#4.11_Split)    
  * [4.12 Summary](#4.12_Summary)


## 4.2 Introduction<a id='4.2_Introduction'></a>

This notebook comprehensively outlines the process of preparing our dataset for machine learning applications. The workflow includes importing the necessary libraries, loading and exploring the data, conducting feature engineering, encoding categorical variables, scaling numerical features, generating new features, selecting relevant features, and finally, splitting the dataset into training and testing sets. Each step is designed to ensure that the dataset is optimized and ready for building robust and effective machine learning models.

## 4.3 Imports<a id='4.3_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

#from library.sb_utils import save_file

## 4.4 Load Data<a id='4.4_Load_Data'></a>

In [2]:
churn_data = pd.read_csv('churn_data_step3_features.csv')
churn_data.head().T

,0,1,2,3,4
Gender,0,1,1,1,0
Senior Citizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
Tenure,1,34,2,45,2
Phone Service,No,Yes,Yes,No,Yes
Multiple Lines,No phone service,No,No,No phone service,No
Internet Service,DSL,DSL,DSL,DSL,Fiber optic
Online Security,No,Yes,Yes,Yes,No
Online Backup,Yes,No,Yes,No,No


## 4.5 Explore the Data<a id='4.5_Explore_Data'></a>

In [3]:
churn_data.describe()

,Gender,Senior Citizen,Tenure,Monthly Charges,Total Charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.504756,0.162147,32.371149,64.761692,2283.300441
std,0.500013,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,0.000000,18.250000,18.800000
25%,0.000000,0.000000,9.000000,35.500000,401.450000
50%,1.000000,0.000000,29.000000,70.350000,1397.475000
75%,1.000000,0.000000,55.000000,89.850000,3794.737500
max,1.000000,1.000000,72.000000,118.750000,8684.800000


In [4]:
churn_data.shape

(7043, 20)

In [5]:
churn_data.isnull().sum()

Gender                 0
Senior Citizen         0
Partner                0
Dependents             0
Tenure                 0
Phone Service          0
Multiple Lines         0
Internet Service       0
Online Security        0
Online Backup          0
Device Protection      0
Tech Support           0
Streaming TV           0
Streaming Movies       0
Contract               0
Paperless Billing      0
Payment Method         0
Monthly Charges        0
Total Charges         11
Churn                544
dtype: int64

In [6]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             7043 non-null   int64  
 1   Senior Citizen     7043 non-null   int64  
 2   Partner            7043 non-null   object 
 3   Dependents         7043 non-null   object 
 4   Tenure             7043 non-null   int64  
 5   Phone Service      7043 non-null   object 
 6   Multiple Lines     7043 non-null   object 
 7   Internet Service   7043 non-null   object 
 8   Online Security    7043 non-null   object 
 9   Online Backup      7043 non-null   object 
 10  Device Protection  7043 non-null   object 
 11  Tech Support       7043 non-null   object 
 12  Streaming TV       7043 non-null   object 
 13  Streaming Movies   7043 non-null   object 
 14  Contract           7043 non-null   object 
 15  Paperless Billing  7043 non-null   object 
 16  Payment Method     7043 

In [7]:
for x in churn_data.columns:
    print(f'{x} has value counts -- {churn_data[x].nunique()}')

Gender has value counts -- 2
Senior Citizen has value counts -- 2
Partner has value counts -- 2
Dependents has value counts -- 2
Tenure has value counts -- 73
Phone Service has value counts -- 2
Multiple Lines has value counts -- 3
Internet Service has value counts -- 3
Online Security has value counts -- 3
Online Backup has value counts -- 3
Device Protection has value counts -- 3
Tech Support has value counts -- 3
Streaming TV has value counts -- 3
Streaming Movies has value counts -- 3
Contract has value counts -- 3
Paperless Billing has value counts -- 2
Payment Method has value counts -- 4
Monthly Charges has value counts -- 1585
Total Charges has value counts -- 6530
Churn has value counts -- 2


In [8]:
# Copy churn_data to a new DataFrame df
df = churn_data.copy()

In [9]:
unique_values_partner = df['Payment Method'].unique()
print(unique_values_partner)

['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


## 4.6 Handle missing values<a id='4.6_missing'></a>

In [10]:
# Check for missing values
print(churn_data.isnull().sum())

Gender                 0
Senior Citizen         0
Partner                0
Dependents             0
Tenure                 0
Phone Service          0
Multiple Lines         0
Internet Service       0
Online Security        0
Online Backup          0
Device Protection      0
Tech Support           0
Streaming TV           0
Streaming Movies       0
Contract               0
Paperless Billing      0
Payment Method         0
Monthly Charges        0
Total Charges         11
Churn                544
dtype: int64


**Missing Values:** 'Total Charges' has 11 missing values. 'Churn' has 544 missing values. Numerical Variables: The columns tenure, Monthly Charges, and Total Charges have numerical values and many of the columns are categorical.

### 4.6.1 Total Charges missing values<a id='4.7.1_Replace'></a>

In [11]:
churn_data[churn_data['Total Charges'].isna()]

,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
488,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,1,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,0,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,0,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,0,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,NaN


The analysis shows that 'Total Charges' is NaN when 'Tenure' is 0. This means customers with 0 tenure haven't yet been charged. The next step is to replace these NaN values with 0.

##### 4.6.1.1 Replace missing values <a id='4.6.1.1_Replace'></a>

In [12]:
Missing_TotalCharges = churn_data[churn_data['Total Charges'].isna()]

churn_data.loc[churn_data['Tenure'] == 0, 'Total Charges'] = 0

churn_data[churn_data['Tenure']== 0]

,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
488,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,0.0,No
753,1,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,0.0,No
936,0,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,0.0,No
1082,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,0.0,No
1340,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,0.0,No
3331,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,0.0,No
3826,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,0.0,No
4380,0,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,0.0,No
5218,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,0.0,No
6670,0,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,0.0,NaN


In [13]:
print(len(Missing_TotalCharges), "rows affected")

11 rows affected


There are no longer any missing values.

## 4.7 Encode categorical values<a id='4.7_Encoding'></a>

### 4.7.1 Replace 'Yes', 'No' values<a id='4.7.1_Replace_Yes_No'></a>

In [14]:
# List of columns to convert
columns = ['Partner', 'Dependents', 'Phone Service', 'Paperless Billing', 'Churn']

# Replace 'Yes' with 1 and 'No' with 0 and 'No internet service' with 0
df[columns] = df[columns].replace({'Yes': 1, 'No': 0})

columns = ['Partner', 'Dependents', 'Phone Service', 'Paperless Billing']

# Convert datatype to integer 
df[columns] = df[columns].astype(int)

df.head()

,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,0,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0.0
1,1,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0.0
2,1,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1.0
3,1,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0.0
4,0,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1.0


### 4.7.2 Encode ordinal variables<a id='4.7.2_ordinal'></a>

In [15]:
unique_values_partner = df['Contract'].unique()
print(unique_values_partner)

['Month-to-month' 'One year' 'Two year']


### 4.7.3 Encode other Categorical Variables<a id='4.7.3_Other'></a>

In [16]:
# Encode 'Contract' from categorical to numerical
contract_column = 'Contract'

# Replace 'Month-to-month' with 0, 'One year' with 1, and 'Two year' with 2 in the 'Contract' column
df[contract_column] = df[contract_column].replace({'Month-to-month': 0, 'One year': 1, 'Two year': 2})

# Convert datatype to integer
df[contract_column] = df[contract_column].astype(int)

# Display the first few rows of the dataframe to check the changes
print(df.head())

   Gender  Senior Citizen  Partner  Dependents  Tenure  Phone Service  \
0       0               0        1           0       1              0   
1       1               0        0           0      34              1   
2       1               0        0           0       2              1   
3       1               0        0           0      45              0   
4       0               0        0           0       2              1   

     Multiple Lines Internet Service Online Security Online Backup  \
0  No phone service              DSL              No           Yes   
1                No              DSL             Yes            No   
2                No              DSL             Yes           Yes   
3  No phone service              DSL             Yes            No   
4                No      Fiber optic              No            No   

  Device Protection Tech Support Streaming TV Streaming Movies  Contract  \
0                No           No           No               No  

In [17]:
# List of specific columns
columns = ['Internet Service', 'Payment Method', 'Multiple Lines', 'Online Security' , 
           'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies']

# Check and clean column names (if necessary)
df.columns = df.columns.str.strip()

# Get unique values for each column and print them
for column in columns:
    if column in df.columns:
        unique_values = df[column].unique()
        print(f"Unique values in the '{column}' column:")
        print(unique_values)
        print()  # Print a newline for better readability
    else:
        print(f"Column '{column}' not found in the DataFrame.")

Unique values in the 'Internet Service' column:
['DSL' 'Fiber optic' 'No']

Unique values in the 'Payment Method' column:
['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']

Unique values in the 'Multiple Lines' column:
['No phone service' 'No' 'Yes']

Unique values in the 'Online Security' column:
['No' 'Yes' 'No internet service']

Unique values in the 'Online Backup' column:
['Yes' 'No' 'No internet service']

Unique values in the 'Device Protection' column:
['No' 'Yes' 'No internet service']

Unique values in the 'Tech Support' column:
['No' 'Yes' 'No internet service']

Unique values in the 'Streaming TV' column:
['No' 'Yes' 'No internet service']

Unique values in the 'Streaming Movies' column:
['No' 'Yes' 'No internet service']



In [18]:
#columns = ['Internet Service', 'Contract', 'Payment Method']

# Perform one-hot encoding on the specified columns
df = pd.get_dummies(df, columns=columns, drop_first=True)

df.head()

,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Contract,Paperless Billing,Monthly Charges,Total Charges,...,Online Backup_No internet service,Online Backup_Yes,Device Protection_No internet service,Device Protection_Yes,Tech Support_No internet service,Tech Support_Yes,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes
0,0,0,1,0,1,0,0,1,29.85,29.85,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,34,1,1,0,56.95,1889.50,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,2,1,0,1,53.85,108.15,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,45,0,1,0,42.30,1840.75,...,0,0,0,1,0,1,0,0,0,0
4,0,0,0,0,2,1,0,1,70.70,151.65,...,0,0,0,0,0,0,0,0,0,0


## 4.8 Scale numerical features <a id='4.8_Scaling'></a>

In [19]:
#Feature scaling
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Scale the numerical features
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
numeric_features = [col for col in numeric_features if col != 'Churn']

df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Display the first few rows of the dataframe after scaling
print(df.head())

     Gender  Senior Citizen  Partner  Dependents    Tenure  Phone Service  \
0 -1.009559       -0.439916        1           0 -1.277445              0   
1  0.990532       -0.439916        0           0  0.066327              1   
2  0.990532       -0.439916        0           0 -1.236724              1   
3  0.990532       -0.439916        0           0  0.514251              0   
4 -1.009559       -0.439916        0           0 -1.236724              1   

   Contract  Paperless Billing  Monthly Charges  Total Charges  ...  \
0         0                  1        -1.160323      -0.994194  ...   
1         1                  0        -0.259629      -0.173740  ...   
2         0                  1        -0.362660      -0.959649  ...   
3         1                  0        -0.746535      -0.195248  ...   
4         0                  1         0.197365      -0.940457  ...   

   Online Backup_No internet service  Online Backup_Yes  \
0                                  0               

## 4.9 Create new features <a id='4.9_Create'></a>

In [20]:
#Create new features 
from sklearn.preprocessing import PolynomialFeatures

# Create interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
interaction_terms = poly.fit_transform(df[numeric_features])

# Convert the interaction terms to a DataFrame
interaction_df = pd.DataFrame(interaction_terms, columns=poly.get_feature_names(numeric_features))

# Merge the interaction terms back into the original dataframe
df = pd.concat([df, interaction_df], axis=1)

# Display the first few rows of the dataframe after adding interaction terms
print(df.head())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 4.10 Feature selection <a id='4.10_Feature'></a>

In [ ]:
#Feature selection (removing highly correlated features)

# Calculate the correlation matrix
correlation_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = correlation_matrix.where(pd.np.triu(pd.np.ones(correlation_matrix.shape), k=1).astype(pd.np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop highly correlated features
df.drop(to_drop, axis=1, inplace=True)

# Display the first few rows of the dataframe after feature selection
print(df.head())

## 4.11 Train/Test Split<a id='4.11_Split'></a>

In [ ]:
# Filter the DataFrame to only include rows where 'Churn' is not NaN
df_notna = df[df['Churn'].notna()]

# Separate features (X) and target (y)
X = df_notna.drop('Churn', axis=1)
y = df_notna['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
#Check the `dtypes` attribute of `X_train` to verify all features are numeric
X_train.dtypes

In [ ]:
X_test.dtypes

## 4.12 Summary<a id='4.12_Summary'></a>

In this notebook, we systematically prepare a dataset for machine learning applications through a series of detailed steps. We begin by importing essential libraries required for data manipulation, visualization, and machine learning. Next, we load and explore the churn dataset, performing initial exploratory data analysis to understand its structure and content. We then proceed with feature engineering, where we encode categorical variables, scale numerical features, and generate new interaction terms to capture potential non-linear relationships in the data. Additionally, we conduct feature selection to remove highly correlated features that could negatively impact model performance. Finally, we split the dataset into training and testing sets based on the presence of NaN values in the 'Churn' column. This comprehensive approach ensures that the dataset is clean, well-structured, and optimized for building robust and effective machine learning models.